<a href="https://colab.research.google.com/github/jperdomos/Parcial2/blob/main/Parcial_SyS_2024_2_Montado_en_DashBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install -U yt-dlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00
     / 2.7 MB 9.4 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile app.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Andrés Fernando Sospanta López")
    st.write("- Juan Camilo Perdomo Soto")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 1: Expresión del Espectro de Fourier",
                          "Punto 2: Simulación y Espectro de Fourier",
                          "Punto 3: Modulación en Amplitud (AM) con Audio",
                          "Punto 4: THD y Factor de Potencia"))

# Parámetros generales
F_o = 10  # Frecuencia fundamental
ti = -1 / (2 * F_o)  # Tiempo inicial
tf = 1 / (2 * F_o)   # Tiempo final
T = tf - ti          # Periodo total
wo = 2 * np.pi / T   # Frecuencia angular
Fs = 100 * F_o       # Frecuencia de muestreo

tv = np.linspace(ti, tf, 1000)
A = 1
x = np.abs(A * np.sin(2 * np.pi * F_o * tv))**2

# Mostrar contenido según el punto seleccionado
if punto == "Punto 1: Expresión del Espectro de Fourier":
    st.write("### Punto 1: Expresión del Espectro de Fourier (Forma Exponencial y Trigonométrica)")

    # Desarrollo de la señal
    st.write("#### Desarrollo:")
    st.write("Dada la señal original:")
    st.latex(r"x(t) = |A \sin(2\pi F_o t)|^2 = A^2 \sin^2(2\pi F_o t) = A^2 \left( \frac{1}{2} - \frac{\cos(2\pi 2 F_o t)}{2} \right)")

    st.write("Aplicando la serie trigonométrica:")
    st.latex(r"x(t) = a_0 + \sum_{n=-N}^N a_n \cos(n\omega_0 t) + b_n \sin(n\omega_0 t)")
    st.write("Como la señal es par, \(b_n = 0\) para todos los \(n\). Los coeficientes son:")
    st.latex(r"a_0 = \frac{A^2}{2}")
    st.latex(r"a_2 = -\frac{A^2}{2}")
    st.latex(r"a_n = 0 \quad \text{para} \quad n \neq 2")

    st.write("Para la serie exponencial compleja:")
    st.latex(r"x(t) = \sum_{n=-N}^{N} c_n e^{j n \omega_0 t}")
    st.write("Donde:")
    st.latex(r"c_0 = \frac{A^2}{2}")
    st.latex(r"c_2 = -\frac{A^2}{4}")
    st.latex(r"c_{-2} = -\frac{A^2}{4}")

    # Graficar la señal
    st.write("### Gráfico de la señal $|A\\sin(2\\pi F_o t)|^2$")
    fig, ax = plt.subplots()
    ax.plot(tv, x)
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('$x(t)$')
    ax.grid(True)
    st.pyplot(fig)

elif punto == "Punto 2: Simulación y Espectro de Fourier":
    st.write("### Punto 2: Simulación y Espectro de Fourier")
    st.write("""
    #### Desarrollo:
    Ahora calculamos y graficamos el espectro de Fourier en términos de magnitud y fase, y reconstruimos la señal para diferentes valores de \(N\).
    """)

    # Definir bases de Fourier
    N = 10  # Número de armónicos
    wo = 2 * np.pi / T  # Frecuencia fundamental angular
    phin = np.zeros((len(tv), 2 * N + 1), dtype=np.complex_)  # Matriz para las bases
    for n in range(-N, N + 1):
        phin[:, n + N] = np.exp(1j * n * wo * tv)

    # Mostrar las bases de Fourier una a la vez
    st.write("### Bases de Fourier:")
    n_base = st.slider("Selecciona la base de Fourier a mostrar:", -N, N, 0)

    fig, ax = plt.subplots()
    ax.plot(tv, np.real(phin[:, n_base + N]), label=f"$Re(\\phi_{n_base}(t))$")
    ax.plot(tv, np.imag(phin[:, n_base + N]), label=f"$Im(\\phi_{n_base}(t))$")
    ax.set_xlabel("t [s]")
    ax.set_ylabel("Base de Fourier")
    ax.legend()
    ax.grid(True)
    st.pyplot(fig)

    # Calcular el espectro
    cn = np.zeros(2 * N + 1, dtype=np.complex_)
    cn[N] = A**2 / 2  # Nivel DC
    cn[N - 2] = -A**2 / 4  # Armónico -2
    cn[N + 2] = -A**2 / 4  # Armónico 2
    nv = np.linspace(-N, N, 2 * N + 1)  # Vector de armónicos

    # Gráficos del espectro de Fourier
    st.write("### Espectro de Fourier:")
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))

    # Real de Cn
    axs[0, 0].stem(nv, np.real(cn), 'r')
    axs[0, 0].set_xlabel(r'$n\omega_0$ [rad/s]')
    axs[0, 0].set_ylabel(r'$Re\{C_n\}$')
    axs[0, 0].grid()

    # Imaginario de Cn
    axs[0, 1].stem(nv, np.imag(cn), 'r')
    axs[0, 1].set_xlabel(r'$n\omega_0$ [rad/s]')
    axs[0, 1].set_ylabel(r'$Im\{C_n\}$')
    axs[0, 1].grid()

    # Magnitud de Cn
    axs[1, 0].stem(nv, abs(cn), 'r')
    axs[1, 0].set_xlabel(r'$n\omega_0$ [rad/s]')
    axs[1, 0].set_ylabel(r'$|C_n|$')
    axs[1, 0].grid()

    # Fase de Cn
    axs[1, 1].stem(nv, np.angle(cn), 'r')
    axs[1, 1].set_xlabel(r'$n\omega_0$ [rad/s]')
    axs[1, 1].set_ylabel(r'$\langle C_n$')
    axs[1, 1].grid()

    fig.tight_layout()
    st.pyplot(fig)

    # Graficar la reconstrucción de la señal
    st.write("### Reconstrucción de la señal y error relativo")

    def pltest(Na=1):
        ind = np.arange(N - Na, N + Na + 1)
        er = 1 - np.sum(abs(cn[ind])**2) / ((A**2) / 2)  # Error relativo
        xe = phin[:, ind].dot(cn[ind])  # Señal reconstruida

        # Graficar la señal reconstruida y la original
        fig, ax = plt.subplots()
        ax.plot(tv, xe, label='$x_e(t)$', color='b')
        ax.plot(tv, x, label='$x(t)$', color='r')
        ax.set_title(f'Error Relativo = {100 * er:.2f}%')
        ax.set_xlabel('t [s]')
        ax.set_ylabel('x(t)')
        ax.grid(True)
        ax.legend()
        st.pyplot(fig)

    # Interfaz interactiva para cambiar Na
    st.write("Ajusta el número de armónicos \(N\) utilizados para la reconstrucción:")
    Na = st.slider('Número de armónicos', 1, N, 1)
    pltest(Na)

    # Crear frecuencias logarítmicas para el Diagrama de Bode
    frequencies = np.linspace(-N, N, 2 * N + 1) * 2 * np.pi / T
    log_freq = np.logspace(np.log10(np.min(frequencies[frequencies > 0])), np.log10(np.max(frequencies)), num=500)

    # Interpolación de magnitudes y fases
    magnitudes = np.abs(cn)
    phases = np.angle(cn, deg=True)
    mag_interp = interp1d(frequencies, magnitudes, kind='linear', fill_value='extrapolate')
    phase_interp = interp1d(frequencies, phases, kind='linear', fill_value='extrapolate')

    # Interpolación para las frecuencias logarítmicas
    log_magnitudes = mag_interp(log_freq)
    log_phases = phase_interp(log_freq)

    # Graficar el Diagrama de Bode
    st.write("### Diagrama de Bode")

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

    # Diagrama de Bode - Magnitud
    ax1.semilogx(log_freq, 20 * np.log10(log_magnitudes), 'b-', marker='o')  # Magnitud en dB
    ax1.set_title('Diagrama de Bode')
    ax1.set_xlabel('Frecuencia [rad/s]')
    ax1.set_ylabel('Magnitud [dB]')
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Diagrama de Bode - Fase
    ax2.semilogx(log_freq, log_phases, 'r-', marker='o')  # Fase en grados
    ax2.set_xlabel('Frecuencia [rad/s]')
    ax2.set_ylabel('Fase [grados]')
    ax2.grid(True, which='both', linestyle='--', linewidth=0.5)

    plt.tight_layout()
    st.pyplot(fig)

if punto == "Punto 3: Modulación en Amplitud (AM) con Audio":
    st.write("### Punto 3: Modulación en Amplitud (AM) con Audio")
     # Mostrar el desarrollo matemático de la Transformada de Fourier
    st.write("#### Desarrollo matemático de la Transformada de Fourier para la señal modulada:")
    st.latex(r"""
    Y(\omega) = \mathcal{F}\{y(t)\} = \mathcal{F}\left\{\left(1+\frac{m(t)}{A_c}\right)c(t)\right\}
    = \mathcal{F}\{c(t)\}+\frac{1}{A_c}\mathcal{F}\{m(t)c(t)\}
    """)

    st.latex(r"""
    C(\omega) = \mathcal{F}\{c(t)\} = \mathcal{F}\{A_c\sin(2\pi F_c t)\} = A_c\mathcal{F}\left\{\frac{e^{2\pi F_ct}-e^{-2\pi F_ct}}{2j}\right\}
    """)

    st.latex(r"""
    \mathcal{F}\{e^{\pm j\omega_0 t}\} = 2\pi\delta(\omega \mp \omega_0)
    """)

    st.latex(r"""
    \boxed{C(\omega) = \frac{A_c\pi}{j}(\delta(\omega - 2\pi F_c)-\delta(\omega + 2\pi F_c))}
    """)

    st.latex(r"""
    \frac{1}{A_c}\mathcal{F}\{m(t)c(t)\} = \frac{1}{A_c}\mathcal{F}\{m(t)A_c\sin(2\pi F_c t)\} = \mathcal{F}\{m(t)\sin(2\pi F_c t)\}
    = \mathcal{F}\left\{\frac{m(t)e^{2\pi F_ct}-m(t)e^{-2\pi F_ct}}{2j}\right\}
    """)

    st.latex(r"""
    \mathcal{F}\{x(t)e^{\pm j\omega_0t}\} = X(\omega \mp \omega_0)
    """)

    st.latex(r"""
    \boxed{\frac{1}{A_c}\mathcal{F}\{m(t)c(t)\} = \frac{1}{2j}\left(M(\omega - 2\pi F_c)-M(\omega + 2\pi F_c)\right)}
    """)

    st.latex(r"""
    \boxed{Y(\omega) = \frac{A_c\pi}{j}(\delta(\omega - 2\pi F_c)-\delta(\omega + 2\pi F_c)) + \frac{1}{2j}\left(M(\omega - 2\pi F_c)-M(\omega + 2\pi F_c)\right)}
    """)
    st.write("Sube un archivo de audio para la simulación de AM...")

    ## En este punto del parcial tuvimos que reemplazar la parte del link de Youtube por un apartado donde el usuario carga directamente el archivo .mp3 o .wav puesto que
    ## al momento de tratar de implementarlo en ngrok daba error al tratar de hacerlo con el link, sin embargo, en el otro archivo .ipynb esta hecho con el link de Youtube.

    # Permitir al usuario subir un archivo de audio
    uploaded_file = st.file_uploader("Sube un archivo de audio (wav, mp3)", type=["wav", "mp3"])

    if uploaded_file is not None:
        # Leer el archivo de audio
        data, fs = sf.read(uploaded_file)
        st.write(f"Frecuencia de muestreo: {fs} Hz")

        # Seleccionar un fragmento de la señal de 5 segundos
        ti = 20  # Segundo inicial
        tf = 25  # Segundo final
        m = data[int(ti * fs):int(tf * fs), 0]  # Señal mensaje en el primer canal

        # Guardar fragmento en un archivo temporal para reproducirlo
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio_file:
            sf.write(temp_audio_file.name, m, fs)
            st.audio(temp_audio_file.name)

        # Parámetros de la modulación
        mod_index = st.slider("Selecciona el índice de modulación", 0.1, 1.0, 0.5)
        Fc = 15000  # Frecuencia de la portadora
        t = np.linspace(0, len(m) / fs, len(m))  # Vector de tiempo
        Ac = np.max(np.abs(m)) / mod_index  # Amplitud de la portadora ajustada por el índice de modulación
        c = Ac * np.sin(2 * np.pi * Fc * t)  # Señal portadora

        # Graficar señal portadora y señal mensaje en el tiempo
        fig, ax = plt.subplots()
        ax.plot(t, c, label='Portadora')
        ax.plot(t, m, label='Mensaje')
        ax.set_xlabel('Tiempo [s]')
        ax.set_ylabel('Amplitud')
        ax.legend()
        st.pyplot(fig)

        # Guardar la portadora como archivo temporal y reproducirla
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_portadora_file:
            sf.write(temp_portadora_file.name, c, fs)
            st.audio(temp_portadora_file.name)

        # Señal modulada en AM
        y = (1 + m / Ac) * c
        st.write("Gráfico de la señal modulada en AM...")
        fig, ax = plt.subplots()
        ax.plot(t, y)
        ax.set_xlabel('Tiempo [s]')
        ax.set_ylabel('y(t)')
        st.pyplot(fig)

        # Guardar la señal modulada y reproducirla
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_modulada_file:
            sf.write(temp_modulada_file.name, y, fs)
            st.audio(temp_modulada_file.name)

        # Calcular los espectros de Fourier
        st.write("Calculando los espectros de Fourier de la señal mensaje, portadora y modulada...")
        Xfm = np.fft.fft(m)
        Xfc = np.fft.fft(c)
        Xfy = np.fft.fft(y)
        vfre = np.fft.fftfreq(len(c), 1 / fs)

        # Graficar los espectros
        fig, ax = plt.subplots()
        ax.plot(vfre, np.abs(Xfm), label='Mensaje')
        ax.plot(vfre, np.abs(Xfc), label='Portadora')
        ax.plot(vfre, np.abs(Xfy), label='Modulada')
        ax.set_xlabel('Frecuencia (Hz)')
        ax.set_ylabel('Magnitud')
        ax.set_ylim([0, 1.1 * np.max(np.abs(Xfy))])
        ax.legend()
        st.pyplot(fig)

elif punto == "Punto 4: THD y Factor de Potencia":
    st.write("### Punto 4: THD y Factor de Potencia")

    # Selección interactiva de los valores de R y C
    R = st.slider("Selecciona el valor de la resistencia (R) en ohmios", 1, 100, 10)
    C = st.slider("Selecciona el valor de la capacitancia (C) en microfaradios (μF)", 1.0, 1000.0, 100.0)
    C = C * 1e-6

    # Simulación de THD y factor de potencia
    st.write("Simulando THD y factor de potencia para un rectificador de onda completa...")
    # Parámetros de simulación
    Fs = 10000  # Frecuencia de muestreo
    T = 1 / 50  # Periodo de la señal (50 Hz)
    t = np.linspace(0, 2 * T, int(Fs * 2 * T))  # Tiempo de simulación corregido
    f = 50  # Frecuencia fundamental de la señal de entrada

    # Generar la señal de voltaje de entrada (onda seno)
    Vin = np.sin(2 * np.pi * f * t)

    # Simulación de rectificador de onda completa con carga resistiva
    I_R = np.abs(Vin / R)  # Corriente a través de la carga resistiva

    # Simulación de rectificador de onda completa con carga RC
    V_C = np.zeros_like(t)  # Inicializar voltaje en el condensador
    I_RC = np.zeros_like(t)  # Inicializar corriente en la carga RC

    # Simulación del voltaje y la corriente en la carga RC
    for i in range(1, len(t)):
        V_C[i] = V_C[i-1] + (Vin[i-1] - V_C[i-1]) / (R * C) * (t[i] - t[i-1])
        I_RC[i] = np.abs((Vin[i] - V_C[i]) / R)

    # Calcular la FFT de la corriente
    def calculate_fft(signal, Fs):
        N = len(signal)
        fft_values = fft(signal)
        fft_magnitude = 2.0 / N * np.abs(fft_values[:N // 2])
        freqs = np.fft.fftfreq(N, 1 / Fs)[:N // 2]
        return freqs, fft_magnitude

    # Calcular la THD a partir de la FFT
    def calculate_thd(signal, Fs):
        freqs, magnitudes = calculate_fft(signal, Fs)
        fundamental_index = np.argmax(magnitudes)  # Detectar la frecuencia fundamental
        fundamental = magnitudes[fundamental_index]  # Componente de la frecuencia fundamental
        harmonic_sum = np.sqrt(np.sum(magnitudes[fundamental_index+1:]**2))  # Suma de los armónicos
        thd = harmonic_sum / fundamental
        return thd

    # Calcular THD y factor de potencia para ambas configuraciones
    thd_resistive = calculate_thd(I_R, Fs)
    thd_rc = calculate_thd(I_RC, Fs)

    # Factor de potencia con THD (sólo por distorsión, no por desfase)
    def power_factor(thd):
        return 1 / np.sqrt(1 + thd**2)

    pf_resistive = power_factor(thd_resistive)
    pf_rc = power_factor(thd_rc)

    # Graficar las corrientes
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))

    ax1.plot(t, I_R, label='Carga Resistiva (R)')
    ax1.set_title('Corriente para Carga Resistiva (R)')
    ax1.set_xlabel('Tiempo [s]')
    ax1.set_ylabel('Corriente [A]')
    ax1.grid()

    ax2.plot(t, I_RC, label='Carga RC en Serie', color='orange')
    ax2.set_title('Corriente para Carga RC en Serie')
    ax2.set_xlabel('Tiempo [s]')
    ax2.set_ylabel('Corriente [A]')
    ax2.grid()

    fig.tight_layout()
    st.pyplot(fig)

    # FFT para carga resistiva
    freqs_R, fft_magnitude_R = calculate_fft(I_R, Fs)

    # FFT para carga RC
    freqs_RC, fft_magnitude_RC = calculate_fft(I_RC, Fs)

    # Graficar FFT para ambas cargas
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6))

    # Gráfico de FFT para la carga resistiva
    ax1.plot(freqs_R, fft_magnitude_R)
    ax1.set_title('Espectro de Fourier - Carga Resistiva (R)')
    ax1.set_xlabel('Frecuencia [Hz]')
    ax1.set_ylabel('Magnitud')
    ax1.grid()

    # Gráfico de FFT para la carga RC
    ax2.plot(freqs_RC, fft_magnitude_RC, color='orange')
    ax2.set_title('Espectro de Fourier - Carga RC en Serie')
    ax2.set_xlabel('Frecuencia [Hz]')
    ax2.set_ylabel('Magnitud')
    ax2.grid()

    fig.tight_layout()
    st.pyplot(fig)

    # Mostrar los resultados de THD y factor de potencia
    st.write(f"**THD para carga resistiva**: {thd_resistive * 100:.2f}%")
    st.write(f"**Factor de potencia para carga resistiva**: {pf_resistive:.2f}")
    st.write(f"**THD para carga RC**: {thd_rc * 100:.2f}%")
    st.write(f"**Factor de potencia para carga RC**: {pf_rc:.2f}")



Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLk0AGPalJtBmv5rmZikOqohYo_MQku3xzZZgzePSJ7crCX"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 8501
!nohup streamlit run app.py --server.port 8501 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(8501)
print(f"Aplicación aquí: {public_url}")



nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://f49d-34-172-233-10.ngrok-free.app" -> "http://localhost:8501"
